<a href="https://colab.research.google.com/github/JaimieOnigkeit/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Jaimie_Onigkeit_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-02-17 22:14:13--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-02-17 22:14:13 (287 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [3]:
#I admit, I copied this cell from the lecture notebook.
df = pd.read_csv('house-votes-84.data', names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [4]:
#Check data types to make sure tests can be performed
df.dtypes

party                   object
handicapped-infants     object
water-project           object
budget                  object
physician-fee-freeze    object
el-salvador-aid         object
religious-groups        object
anti-satellite-ban      object
aid-to-contras          object
mx-missile              object
immigration             object
synfuels                object
education               object
right-to-sue            object
crime                   object
duty-free               object
south-africa            object
dtype: object

In [47]:
#Can't do math on objects
df = df.replace({'y':1, 'n':0, '?':np.NaN})

print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [21]:
#Separate republican data out
rep = df[df['party']=='republican']

rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [22]:
#Seperate democrat data out
dem = df[df['party']=='democrat']

dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


# Tests

I'm going to compare each categoryof votes. For each, the null hypothesis will be that republicans and democrats agree on the measure.

In [0]:
#I'm pretty lazy when it comes to typing, so I'm going to make a function.

def compare (column):
  sample1 = dem[column]
  sample2 = rep[column]

  return stats.ttest_ind(sample1, sample2, nan_policy = 'omit')

In [28]:
column = ['education']

compare(column)

Ttest_indResult(statistic=masked_array(data=[-20.500685724563073],
             mask=[False],
       fill_value=1e+20), pvalue=masked_array(data=1.8834204e-64,
             mask=False,
       fill_value=1e+20))

Yay! It works! I compared the output to the test in the lecture notebook, and it matches!

In [31]:
#Can I do them all at once?

columns = ['handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

compare(columns)

Ttest_indResult(statistic=masked_array(data=[9.205264294809222, -0.08896538137868286,
                   23.21277691701378, -49.36708157301406,
                   -21.13669261173219, -9.737575825219457,
                   12.526187929077842, 18.052093200819733,
                   16.437503268542994, -1.7359117329695164,
                   8.293603989407588, -20.500685724563073,
                   -13.51064251060933, -16.342085656197696,
                   12.853146132542978, 6.849454815841208],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False],
       fill_value=1e+20), pvalue=masked_array(data=[1.61344033e-018, 9.29155682e-001, 2.07034028e-077,
                   1.99426231e-177, 5.60052011e-068, 2.39367225e-020,
                   8.52103302e-031, 2.82471841e-054, 5.03079265e-047,
                   8.33024849e-002, 1.57593223e-015, 1.88342040e-064,
                   1.22785817e-034, 

In [32]:
#I wonder if I can put that in a data frame...

vote_data = compare(columns)

vote_data

Ttest_indResult(statistic=masked_array(data=[9.205264294809222, -0.08896538137868286,
                   23.21277691701378, -49.36708157301406,
                   -21.13669261173219, -9.737575825219457,
                   12.526187929077842, 18.052093200819733,
                   16.437503268542994, -1.7359117329695164,
                   8.293603989407588, -20.500685724563073,
                   -13.51064251060933, -16.342085656197696,
                   12.853146132542978, 6.849454815841208],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False],
       fill_value=1e+20), pvalue=masked_array(data=[1.61344033e-018, 9.29155682e-001, 2.07034028e-077,
                   1.99426231e-177, 5.60052011e-068, 2.39367225e-020,
                   8.52103302e-031, 2.82471841e-054, 5.03079265e-047,
                   8.33024849e-002, 1.57593223e-015, 1.88342040e-064,
                   1.22785817e-034, 

In [34]:
vote_data_df = pd.DataFrame(data = vote_data)

vote_data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9.205264e+00,-0.088965,2.321278e+01,-4.936708e+01,-2.113669e+01,-9.737576e+00,1.252619e+01,1.805209e+01,1.643750e+01,-1.735912,8.293604e+00,-2.050069e+01,-1.351064e+01,-1.634209e+01,1.285315e+01,6.849455e+00
1,1.613440e-18,0.929156,2.070340e-77,1.994262e-177,5.600520e-68,2.393672e-20,8.521033e-31,2.824718e-54,5.030793e-47,0.083302,1.575932e-15,1.883420e-64,1.227858e-34,9.952343e-47,5.997697e-32,3.652674e-11


In [35]:
#Okay, that's working. Now I need to transpose the dataframe and add a column with the issue names

vote_data_df = vote_data_df.T

vote_data_df.head()

,0,1
0,9.205264,1.613440e-18
1,-0.088965,9.291557e-01
2,23.212777,2.070340e-77
3,-49.367082,1.994262e-177
4,-21.136693,5.600520e-68


In [36]:
vote_data_df['issue'] = columns

vote_data_df.head()

,0,1,issue
0,9.205264,1.613440e-18,handicapped-infants
1,-0.088965,9.291557e-01,water-project
2,23.212777,2.070340e-77,budget
3,-49.367082,1.994262e-177,physician-fee-freeze
4,-21.136693,5.600520e-68,el-salvador-aid


In [43]:
vote_data_df.columns=['t-stat', 'p-value','issue']

vote_data_df.head()

,t-stat,p-value,issue
0,9.205264,1.613440e-18,handicapped-infants
1,-0.088965,9.291557e-01,water-project
2,23.212777,2.070340e-77,budget
3,-49.367082,1.994262e-177,physician-fee-freeze
4,-21.136693,5.600520e-68,el-salvador-aid


In [45]:
vote_data_df = vote_data_df[['issue', 't-stat', 'p-value']]

vote_data_df.head()

,issue,t-stat,p-value
0,handicapped-infants,9.205264,1.613440e-18
1,water-project,-0.088965,9.291557e-01
2,budget,23.212777,2.070340e-77
3,physician-fee-freeze,-49.367082,1.994262e-177
4,el-salvador-aid,-21.136693,5.600520e-68


In [55]:
rep_mean = pd.DataFrame(rep.mean())
rep_mean.head()

,0
handicapped-infants,0.187879
water-project,0.506757
budget,0.134146
physician-fee-freeze,0.987879
el-salvador-aid,0.951515


In [56]:
rep_mean.reset_index(inplace=True)
rep_mean.columns=['issue', 'rep_mean']

rep_mean.head()

,issue,rep_mean
0,handicapped-infants,0.187879
1,water-project,0.506757
2,budget,0.134146
3,physician-fee-freeze,0.987879
4,el-salvador-aid,0.951515


In [0]:
dem_mean = pd.DataFrame(dem.mean())

dem_mean

In [57]:
dem_mean.reset_index(inplace=True)
dem_mean.columns=['issue', 'dem_mean']

dem_mean.head()

,issue,dem_mean
0,handicapped-infants,0.604651
1,water-project,0.502092
2,budget,0.888462
3,physician-fee-freeze,0.054054
4,el-salvador-aid,0.215686


In [0]:
means = pd.merge(rep_mean, dem_mean, on='issue', how='inner')

means

In [60]:
final = pd.merge(vote_data_df, means, on='issue', how='inner')

final

,issue,t-stat,p-value,rep_mean,dem_mean
0,handicapped-infants,9.205264,1.613440e-18,0.187879,0.604651
1,water-project,-0.088965,9.291557e-01,0.506757,0.502092
2,budget,23.212777,2.070340e-77,0.134146,0.888462
3,physician-fee-freeze,-49.367082,1.994262e-177,0.987879,0.054054
4,el-salvador-aid,-21.136693,5.600520e-68,0.951515,0.215686
5,religious-groups,-9.737576,2.393672e-20,0.897590,0.476744
6,anti-satellite-ban,12.526188,8.521033e-31,0.240741,0.772201
7,aid-to-contras,18.052093,2.824718e-54,0.152866,0.828897
8,mx-missile,16.437503,5.030793e-47,0.115152,0.758065
9,immigration,-1.735912,8.330248e-02,0.557576,0.471483


Okay, I'm happy with that. Now to look for the specific items requested.

# Goals




### Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

There are eight issues that meet this criteria:

handicapped infants, budget, anti-satelite ban, aid to contras, mx missile, synfuels, duty-free, and south africa

What this means is that for these issues, we reject the null hypothesis because the p-value is less than .05, and that the mean vote was higher for democrats than republicans.

### Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

There are six issues that meet this criteria:

physician fee freeze, el salvador aid, religious groups, education, right to sue, crime

What this means is that for these issues, we reject the null hypothesis because the p-value is less than .05, and that the mean vote was higher for republicans than democrats.

### Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

There are two: 

First, the water project has a p-value of 0.93, which is actually quite a bit higher than .05, so we would fail to reject the null hypothesis in this instance.

Second, the immigration issue has a p-value of 0.083, which is also greater than .05, so we would fail to reject the null hypothesis on this issue as well.